In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
train = pd.read_csv('featuresTrain3.csv')
test_entrenamiento = pd.read_csv('featuresTest3.csv')
test = pd.read_csv('test.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 28 columns):
id                            240000 non-null int64
ciudad                        240000 non-null object
provincia                     240000 non-null object
antiguedad                    240000 non-null float64
habitaciones                  240000 non-null float64
garages                       240000 non-null float64
banos                         240000 non-null float64
metroscubiertos               240000 non-null float64
metrostotales                 240000 non-null float64
idzona                        240000 non-null float64
lat                           240000 non-null float64
lng                           240000 non-null float64
precio                        240000 non-null float64
anio                          240000 non-null float64
mes                           240000 non-null float64
dia                           240000 non-null float64
datoBienCargado              

In [5]:
train1 = train.drop(['id','precio','ciudad','provincia'],axis='columns')
train1.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,anio,...,peso_IDF_titulo,metrosdescubiertos,tipodepropiedad_cb,caracteristicas_cb,provincia_cb,piscina,gimnasio,usosmultiples,escuelascercanas,centroscomercialescercanos
0,2.210044,2.0,1.0,2.0,4.394449,4.394449,23533.0,20.696818,-99.509221,2015.0,...,0.000000,0.000000,0.478679,0.390827,0.413880,0.0,0.0,0.0,0.0,0.0
1,2.397895,3.0,2.0,2.0,5.594711,5.198497,24514.0,19.310205,-99.227655,2013.0,...,0.165371,0.000000,0.386072,0.475439,0.413880,0.0,0.0,0.0,1.0,1.0
2,1.791759,3.0,2.0,2.0,4.976734,5.117994,48551.0,20.696818,-99.509221,2015.0,...,0.033601,3.135494,0.426561,0.390827,0.407525,0.0,0.0,0.0,0.0,0.0
3,0.693147,2.0,1.0,1.0,4.158883,4.219508,53666.0,19.301890,-99.688015,2012.0,...,0.033601,1.609438,0.426561,0.475439,0.420146,0.0,0.0,0.0,1.0,1.0
4,2.397895,2.0,1.0,1.0,4.564348,4.564348,47835.0,20.696818,-99.509221,2016.0,...,0.000000,0.000000,0.478679,0.390827,0.407525,0.0,0.0,0.0,0.0,0.0


In [6]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 24 columns):
antiguedad                    240000 non-null float64
habitaciones                  240000 non-null float64
garages                       240000 non-null float64
banos                         240000 non-null float64
metroscubiertos               240000 non-null float64
metrostotales                 240000 non-null float64
idzona                        240000 non-null float64
lat                           240000 non-null float64
lng                           240000 non-null float64
anio                          240000 non-null float64
mes                           240000 non-null float64
dia                           240000 non-null float64
datoBienCargado               240000 non-null int64
peso_IDF_descripcion          240000 non-null float64
peso_IDF_titulo               240000 non-null float64
metrosdescubiertos            240000 non-null float64
tipodepropiedad_cb         

In [7]:
def normalize(df):
     result = df.copy()
     for feature_name in df.columns:
         max_value = df[feature_name].max()
         min_value = df[feature_name].min()
         result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
     return result

train_entrenamiento=normalize(train1)
train_entrenamiento.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,anio,...,peso_IDF_titulo,metrosdescubiertos,tipodepropiedad_cb,caracteristicas_cb,provincia_cb,piscina,gimnasio,usosmultiples,escuelascercanas,centroscomercialescercanos
0,0.502917,0.272727,0.333333,0.333333,0.489369,0.489369,0.000470,0.661093,0.106726,0.75,...,0.000000,0.000000,0.687922,0.405199,0.349686,0.0,0.0,0.0,0.0,0.0
1,0.545665,0.363636,0.666667,0.333333,0.851528,0.731977,0.000490,0.653553,0.107866,0.25,...,0.067646,0.000000,0.545397,0.754402,0.349686,0.0,0.0,0.0,1.0,1.0
2,0.407732,0.363636,0.666667,0.333333,0.665064,0.707687,0.000971,0.661093,0.106726,0.75,...,0.013745,0.519716,0.607711,0.405199,0.338902,0.0,0.0,0.0,0.0,0.0
3,0.157732,0.272727,0.333333,0.000000,0.418291,0.436584,0.001073,0.653508,0.106002,0.00,...,0.013745,0.266769,0.607711,0.754402,0.360319,0.0,0.0,0.0,1.0,1.0
4,0.545665,0.272727,0.333333,0.000000,0.540633,0.540633,0.000956,0.661093,0.106726,1.00,...,0.000000,0.000000,0.687922,0.405199,0.338902,0.0,0.0,0.0,0.0,0.0


In [8]:
trainlabel = train['precio']
trainlabel.head()

0    2273000.0
1    3600000.0
2    1200000.0
3     650000.0
4    1150000.0
Name: precio, dtype: float64

In [9]:
# layer = best_hiper_parameters[0]
#cargo datos al modelo
clf = MLPRegressor(activation='relu',solver='lbfgs', alpha=1e-5,hidden_layer_sizes=1,batch_size='auto', random_state=1)
clf.fit(train1, trainlabel)    
#return clf

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=1, learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [10]:
parameters_for_testing = {
#    'colsample_bytree':[0.4,0.6,0.8],
#    'gamma':[0,0.03,0.1],
#    'min_child_weight':[1.5,6,10],
#    'learning_rate':[0.1,0.07,0.01],
#    'max_depth':[3,5,7],
#    'n_estimators':[10,100,1000],
#    'reg_alpha':[1e-5, 1e-2,  0.75],
#    'reg_lambda':[1e-5, 1e-2, 0.45],
#    'subsample':[0.6,0.95]
    'hidden_layer_sizes':[1,2,3,4,5,10,15,20,50],
    'random_state':[1,2,5,8,15],
}

In [11]:
test_entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 27 columns):
id                            60000 non-null int64
ciudad                        60000 non-null object
provincia                     60000 non-null object
antiguedad                    60000 non-null float64
habitaciones                  60000 non-null float64
garages                       60000 non-null float64
banos                         60000 non-null float64
metroscubiertos               60000 non-null float64
metrostotales                 60000 non-null float64
idzona                        60000 non-null float64
lat                           60000 non-null float64
lng                           60000 non-null float64
anio                          60000 non-null float64
mes                           60000 non-null float64
dia                           60000 non-null float64
datoBienCargado               60000 non-null int64
peso_IDF_descripcion          60000 non-null floa

In [12]:
test_entrenamiento.head()

,id,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,...,peso_IDF_titulo,metrosdescubiertos,tipodepropiedad_cb,caracteristicas_cb,provincia_cb,piscina,gimnasio,usosmultiples,escuelascercanas,centroscomercialescercanos
0,4941,Miguel Hidalgo,Distrito Federal,3.401197,3.0,1.546874,4.0,5.707110,5.180463,2.423468e+06,...,0.033601,3.031614,0.426768,0.392836,0.404545,0.0,0.0,0.0,0.0,0.0
1,51775,Mérida,Yucatán,2.210044,1.0,1.000000,1.0,4.219508,4.219508,1.138510e+05,...,0.000000,0.000000,0.480004,0.392836,0.567188,0.0,0.0,0.0,0.0,0.0
2,115253,Coyoacán,Distrito Federal,0.000000,2.0,1.000000,2.0,4.477337,4.615121,2.362000e+04,...,0.638628,2.639057,0.480004,0.470865,0.404545,0.0,0.0,0.0,0.0,1.0
3,299321,Acapulco de Juárez,Guerrero,1.098612,2.0,2.000000,2.0,4.465908,4.465908,1.293470e+05,...,0.091081,0.000000,0.480004,0.392836,0.364473,0.0,0.0,0.0,0.0,0.0
4,173570,Tultitlán,Edo. de México,2.397895,2.0,1.000000,1.0,4.394449,4.343805,5.712500e+04,...,0.033601,0.000000,0.426768,0.475463,0.422059,0.0,0.0,0.0,1.0,1.0


In [18]:
test_entrenamiento1 = test_entrenamiento.drop(['id','ciudad','provincia'],axis='columns')

In [20]:
test_entrenamiento1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 24 columns):
antiguedad                    60000 non-null float64
habitaciones                  60000 non-null float64
garages                       60000 non-null float64
banos                         60000 non-null float64
metroscubiertos               60000 non-null float64
metrostotales                 60000 non-null float64
idzona                        60000 non-null float64
lat                           60000 non-null float64
lng                           60000 non-null float64
anio                          60000 non-null float64
mes                           60000 non-null float64
dia                           60000 non-null float64
datoBienCargado               60000 non-null int64
peso_IDF_descripcion          60000 non-null float64
peso_IDF_titulo               60000 non-null float64
metrosdescubiertos            60000 non-null float64
tipodepropiedad_cb            60000 non-null 

In [21]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

test_entrenamiento1=normalize(test_entrenamiento1)
test_entrenamiento1.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,anio,...,peso_IDF_titulo,metrosdescubiertos,tipodepropiedad_cb,caracteristicas_cb,provincia_cb,piscina,gimnasio,usosmultiples,escuelascercanas,centroscomercialescercanos
0,0.773976,0.363636,0.515625,1.000000,0.885443,0.726536,0.048465,0.448899,0.099535,0.25,...,0.017649,0.509016,0.544905,0.399015,0.256829,0.0,0.0,0.0,0.0,0.0
1,0.502917,0.181818,0.333333,0.000000,0.436584,0.436584,0.002276,0.462665,0.142006,0.75,...,0.000000,0.000000,0.626239,0.399015,0.557455,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.272727,0.333333,0.333333,0.514379,0.555953,0.000472,0.448256,0.099947,0.75,...,0.335443,0.443104,0.626239,0.614925,0.256829,0.0,0.0,0.0,0.0,1.0
3,0.250000,0.272727,0.666667,0.333333,0.510931,0.510931,0.002586,0.427296,0.096756,0.75,...,0.047841,0.000000,0.626239,0.399015,0.182761,0.0,0.0,0.0,0.0,0.0
4,0.545665,0.272727,0.333333,0.000000,0.489369,0.474088,0.001142,0.450864,0.100060,0.25,...,0.017649,0.000000,0.544905,0.627648,0.289201,0.0,0.0,0.0,1.0,1.0


In [22]:
prediction = clf.predict(test_entrenamiento1)

In [23]:
ids = test['id'].values
dataKaggle = pd.DataFrame(ids ,columns={'id'})
dataKaggle.head(5)

,id
0,4941
1,51775
2,115253
3,299321
4,173570


In [24]:
#Aca agrego la columna con la prediccion
dataKaggle['target'] = prediction
dataKaggle.nsmallest(5,'target')

,id,label
23,92772,0.150423
40,68117,0.150423
41,39648,0.150423
46,126979,0.150423
58,125013,0.150423


In [25]:
dataKaggle['target'] = dataKaggle['target'].map(lambda x: 0.0 if x < 0.0 else x)

In [26]:
dataKaggle.reset_index()

,index,id,label
0,0,4941,1.581578
1,1,51775,6.418791
2,2,115253,6.734478
3,3,299321,5.956434
4,4,173570,2.281845
5,5,30862,9.001881
6,6,244471,9.100578
7,7,127794,9.010997
8,8,71558,8.141240
9,9,218011,5.927413


In [27]:
dataKaggle.nsmallest(5,'target')

,id,label
23,92772,0.150423
40,68117,0.150423
41,39648,0.150423
46,126979,0.150423
58,125013,0.150423


In [28]:
len(dataKaggle)

60000

In [29]:
dataKaggle.to_csv("predictionRedNeuronal.csv", index = False)